In [ ]:
import h2o
h2o.connect()

In [ ]:
import pandas as pd

In [ ]:
titanic_df = pd.read_csv('/Users/avkashchauhan/learn/comcast/titanic_list.csv')

In [ ]:
titanic_df.shape

In [ ]:
titanic_df.count()

In [ ]:
# Converting Pandas Frame to H2O Frame
titanic = h2o.H2OFrame(titanic_df)

In [ ]:
titanic

In [ ]:
# Note: You will see that the following command will not work
# Because it is a H2OFrame
titanic.count()

In [ ]:
# The Other option to import data directly is to use H2O.
titanic_data = h2o.import_file('/Users/avkashchauhan/learn/seattle-workshop/titanic_list.csv')

In [ ]:
titanic_data.shape

In [ ]:
titanic_data.summary

In [ ]:
titanic_data.describe

In [ ]:
titanic_data.table

In [ ]:
titanic_data['pclass'].table()

In [ ]:
titanic_data['age'].table()

In [ ]:
titanic_data['sex'].table()

In [ ]:
# Loading Estimators
from h2o.estimators.glm import H2OGeneralizedLinearEstimator

In [ ]:
# set this to True if interactive (matplotlib) plots are desired
import matplotlib
interactive = False
if not interactive: matplotlib.use('Agg', warn=False)
import matplotlib.pyplot as plt

In [ ]:
titanic_data.col_names

In [ ]:
response = "survived"

In [ ]:
# Selected Columns
# pclass, survived, sex, age, sibsp, parch, fare, embarked 
#

predictors = ["pclass", "sex", "age", "sibsp", "parch", "fare", "embarked"]
# predictors = titanic_data.columns[:-1]

In [ ]:
predictors

In [ ]:
#Now setting factors to specific columns
titanic_data["pclass"] = titanic_data["pclass"].asfactor()
titanic_data["sex"] = titanic_data["sex"].asfactor()
titanic_data["embarked"] = titanic_data["embarked"].asfactor()


In [ ]:
titanic_data.describe()

In [ ]:
# Spliting the data set for training and validation
titanic_train, titanic_valid = titanic_data.split_frame(ratios=[0.9])

In [ ]:
print(titanic_train.shape)
print(titanic_valid.shape)

# Creating GLM Model

In [ ]:
titanic_glm = H2OGeneralizedLinearEstimator(alpha = .25)

In [ ]:
titanic_glm.train(x = predictors, y = response, training_frame = titanic_train, validation_frame = titanic_valid)

In [ ]:
# print the mse for the validation data
print "mse: ", titanic_glm.mse(valid=True)
print "r2 : ", titanic_glm.r2(valid=True)
print "rmse:", titanic_glm.rmse(valid=True)

# Note: Look for titanic_glm.[TAB] for the values you are interested into

# Adding Grid Search now

In [ ]:
# grid over `alpha`
# import Grid Search
from h2o.grid.grid_search import H2OGridSearch


In [2]:
?

Object `H2OGeneralizedLinearEstimator` not found.


In [ ]:
# select the values for `alpha` to grid over
hyper_params = {'alpha': [0, .25, .5, .75, .1]}

In [ ]:
# this example uses cartesian grid search because the search space is small
# and we want to see the performance of all models. For a larger search space use
# random grid search instead: {'strategy': "RandomDiscrete"}
# initialize the GLM estimator
titanic_glm_hype = H2OGeneralizedLinearEstimator()


In [ ]:
# build grid search with previously made GLM and hyperparameters
titanitc_grid = H2OGridSearch(model = titanic_glm_hype, hyper_params = hyper_params,
                     search_criteria = {'strategy': "Cartesian"})


In [ ]:
# train using the grid
titanitc_grid.train(x = predictors, y = response, training_frame = titanic_train, validation_frame = titanic_valid)


In [ ]:
# sort the grid models by mse
titanic_sorted_grid = titanitc_grid.get_grid(sort_by='mse', decreasing=False)
print(titanic_sorted_grid)

In [ ]:
# If you want to sort by r2 then try this
titanic_sorted_grid = titanitc_grid.get_grid(sort_by='r2', decreasing=False)
print(titanic_sorted_grid)

# Adding multiple hyperparameters

In [ ]:
# Now adding alpha and lambda together
hyper_params = {'alpha': [0, .25, .5, .75, .1], 'lambda': [0, .1, .01, .001, .0001]}

In [ ]:
titanic_glm_hype = H2OGeneralizedLinearEstimator()
titanitc_grid = H2OGridSearch(model = titanic_glm_hype, hyper_params = hyper_params,
                     search_criteria = {'strategy': "Cartesian"})
titanitc_grid.train(x = predictors, y = response, training_frame = titanic_train, validation_frame = titanic_valid)


In [ ]:
# If you want to sort by r2 then try this
titanic_sorted_grid = titanitc_grid.get_grid(sort_by='r2', decreasing=False)
print(titanic_sorted_grid)

In [ ]:
h2o.ls()

In [ ]:
h2o.remove_all()

In [ ]:
h2o.ls()

In [ ]:
newmodel = h2o.get_model("gbm-75df84b0-abdd-4896-bf1e-6802bf132325")

In [ ]:
newmodel

In [ ]:
newmodel.varimp()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcdefaults()
fig, ax = plt.subplots()
variables = newmodel._model_json['output']['variable_importances']['variable']
y_pos = np.arange(len(variables))

In [ ]:
scaled_importance = newmodel._model_json['output']['variable_importances']['scaled_importance']
ax.barh(y_pos, scaled_importance, align='center', color='green', ecolor='black')
ax.set_yticks(y_pos)
ax.set_yticklabels(variables)
ax.invert_yaxis()
ax.set_xlabel('Scaled Importance')
ax.set_title('Variable Importance')
plt.show()

In [ ]:
newmodel

In [ ]:
newmodel._model_json['output']['variable_importances']

In [ ]:
newmodel.score_history()

In [ ]:
from h2o.estimators.gbm import H2OGradientBoostingEstimator

In [ ]:
titanic_gbm = H2OGradientBoostingEstimator()

In [ ]:
titanic_gbm.train(x = predictors, y = response, training_frame = titanic_train, 
                  validation_frame = titanic_valid, model_id = "new_GBM_Model")

In [ ]:
myGbm = h2o.get_model("new_GBM_Model")